# Handbook of MRI Pulse Sequences: Section 3.3 Refocusing Pulses

In this notebook we will atempt to understand the basic components of a refocusing pulse.

Note: Select "Run" above to run all cells, or press "shift + enter"

First we need to install required packages. Might take a minute or two:

In [ ]:
!pip install -q numpy matplotlib scipy ipywidgets sycomore

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

import sycomore
from sycomore import units

## Part 1: Excitation and T2 decay of a single spin isochromat

Suppose a spin isochromat has magnetization $M_0$ and resonance offset $\Delta \omega = \omega - \omega_{ref}$, where $\omega_{ref}$ is the angular frequency of the rotating frame. 

Immediately after an excitation pulse (flip angle = $\theta_1$) along the x-axis, the magnetization of the spin isochromat can be described by:

$$\vec{M}(\Delta \omega, t = 0) = M_0 \begin{bmatrix} 0 \\ \sin \theta_1 \\ \sin \theta_2 \end{bmatrix}$$

In general, $\vec{M}$ can have a different Larmor frequency $\omega$ than the rotating frame $\omega_{ref}$, leading to non-zero values of $\Delta \omega$. Because of this resonance offset, $\vec{M}$ precesses about the z-axis as described by:

$$\vec{M}(\Delta \omega, t) = M_0 \begin{bmatrix} 
e^{-t/T_2}\sin\theta_1\sin(\Delta \omega t)  \\ 
e^{-t/T_2}\sin\theta_1\cos(\Delta \omega t)  \\ 
e^{-t/T_1}\cos\theta_1 + (1-e^{-t/T_1}) \end{bmatrix}$$


where the exponential terms account for the $T_1$ and $T_2$ relaxation effects. 

Let's implement this expression. Below, `dw` is the angular frequency shift $\Delta \omega = 2\pi \Delta f$.

In [2]:
def calculate_magnetization(time_ms, species, flip_angle):
    
    # Note: the sycomore packages defines a counterclockwise rotation for positive angles, whereas 
    # Bernstein defined a clockwise rotation for positive angles. Therefore, we need to invert the rotation here. 
    # ref: https://github.com/lamyj/sycomore/blob/master/src/sycomore/isochromat/Model.cpp
    t = np.array(time_ms)

    dw =  - 2 * np.pi * species.delta_omega.convert_to(units.Hz)
    
    E1 = np.exp(-t/species.T1.convert_to(units.ms))
    E2 = np.exp(-t/species.T2.convert_to(units.ms))

    Mx = E2 * np.sin(flip_angle.convert_to(units.rad)) * np.sin(dw*t*1e-3)
    My = E2 * np.sin(flip_angle.convert_to(units.rad)) * np.cos(dw*t*1e-3)
    Mz = E1 * np.cos(flip_angle.convert_to(units.rad)) + (1 - E1)

    return np.stack((Mx, My, Mz), -1)

We will use the expression defined in Bernstein for a frequency shift:


$$ f = \frac{\gamma}{2\pi}B_0(1-\delta)$$

Therefore, 

$$ \Delta f = f_{rot} -f $$ 


Let's implement this expression. Keep in mind $\frac{\gamma}{2\pi} = 42.58\times 10^6$ Hz.

In [3]:
def get_inhomogeneity(B0=3, inhomogeneity_ppm=0.0):

    # Define the gyromagnetic ratio for protons (42.58 MHz/T)
    gyromagnetic_ratio = 42.58e6  # Hz/T

    freq_rot = gyromagnetic_ratio * B0 # resonant frequency 
    freq = gyromagnetic_ratio * B0 * (1 - inhomogeneity_ppm) 

    delta_freq = (freq - freq_rot)*1e-6 # convert from ppm

    return delta_freq*units.Hz 

Below are some convenient functions to define the species of the spin isochromat as well as to modifty/record the magnetization state 

In [4]:
def get_species(T1, T2, delta_omega=0*units.Hz):   
    species = sycomore.Species(T1, T2, delta_omega=delta_omega)
    return species

def update_and_record_magnetization(M, record, t, update=None):
    if update is not None: M = update @ M
    record.append([t.convert_to(units.ms), M[:3] / M[3]])
    return M

Let's simulate a simple excitation-recovery pulse and record the spin behavior.

In [5]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms):
    
    # define species of spin isochromat
    # ---------------------------------

    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species = get_species(T1, T2, delta_omega=-delta_omega)

    # initialize the spin state 
    t = 0 * units.ms
    M = np.array([0, 0, 1, 1])
    record = [[t.convert_to(units.ms), M[:3] / M[3]]]

    # define experiment bloacks
    # -------------------------

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time
    idle = sycomore.bloch.time_interval(species, step_size) 

    # excitation pulse, phase implies it is along x-axis. Vector rotates about x-axis. If flip angle is 90 degrees, vector flip to +y-axis
    flip_angle = flip_angle_deg * units.deg
    pulse = sycomore.bloch.pulse(flip_angle, phase=np.pi*units.rad) 

    # define experiment blocks
    # ------------------------

    # apply pulse
    M = update_and_record_magnetization(M, record, t, update=pulse)

    # run 100 steps 
    for _ in range(100):
        t += step_size
        M = update_and_record_magnetization(M, record, t, update=idle)

    time, magnetization = list(zip(*record))

    magnetization = np.array(magnetization).round(3)

    magnetization_predicted = calculate_magnetization(time, species, flip_angle)

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.plot(time, magnetization[:, 0], label="Species $M_x$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,0], label="Species $M_x$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, magnetization[:, 2], label="Species $M_z$", linestyle='dotted', color='#BF1515')
    plt.plot(time, magnetization_predicted[:,2], label="Species $M_z$ Predicted", linestyle='dashdot', color='#F2668B')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.subplot(122)
    plt.plot(time, magnetization[:, 1], label="Species $M_y$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,1], label="Species $M_y$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.tight_layout()
    plt.show()

As you can see in the figure below, our simulations and theoretical predictions are identical. Feel free to play around with the widget to understand the spin dynamics.

In [6]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Step size (ms)')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms)>

## Part 2: Refocusing Pulse for a single spin isochromat

If a refocusing RF pulse with a flip angle of $\theta_2$ is applied along the y-axis, the magnetization immediately before $\vec{M}(\Delta \omega, \tau_-)$ and immediately after $\vec{M}(\Delta \omega, \tau_+)$ the refocusing pulse are related by a rotation matrix

$$\vec{M}(\Delta \omega, \tau_+) = \begin{bmatrix} 
\cos\theta_2 & 0 & -\sin\theta_2 \\ 
0 & 1 & 0  \\ 
\sin\theta_2 & 0 & \cos\theta_2 \end{bmatrix}\vec{M}(\Delta \omega, \tau_-)$$

Let's implement this operation. Again, keep in mind that `Sycomore` has the opposite clockwise vs counterclockwise direction relative Bernstein. Therefore, we need to reverse the matrix. 

In [7]:
def calculate_magnetization_pre_refocusing(M_, species, flip_angle):

    # Magnetization before refocusing pulse 
    Mx_, My_, Mz_ = M_    

    cos = np.cos(flip_angle.convert_to(units.rad))
    sin = np.sin(flip_angle.convert_to(units.rad)) 
    
    Mx = Mx_ * cos + Mz_ * sin 
    My = My_ 
    Mz = - Mx_ * sin + Mz_ * cos 
    
    return np.stack((Mx, My, Mz), -1)

In [8]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, flip_angle_refocus_deg=180):
    
    # define species of spin isochromat
    # ---------------------------------

    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species = get_species(T1, T2, delta_omega=-delta_omega)

    # initialize the spin state 
    t = 0 * units.ms
    M = np.array([0, 0, 1, 1])
    record = [[t.convert_to(units.ms), M[:3] / M[3]]]

    # define experiment bloacks
    # -------------------------

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time
    idle = sycomore.bloch.time_interval(species, step_size) 

    # excitation pulse, phase implies it is along x-axis. Vector rotates about x-axis. If flip angle is 90 degrees, vector flip to +y-axis
    flip_angle = flip_angle_deg * units.deg
    pulse = sycomore.bloch.pulse(flip_angle, phase=np.pi*units.rad) 

    # define experiment blocks
    # ------------------------

    # apply pulse
    M = update_and_record_magnetization(M, record, t, update=pulse)

    # run 100 steps 
    for _ in range(100):
        t += step_size
        M = update_and_record_magnetization(M, record, t, update=idle)

    time, magnetization = list(zip(*record))
    print(len(time))
    
    if flip_angle_refocus_deg > 0:
        # apply (e.g., 180) pulse along y direction (specified by phase), only spins off-phase rotate above y-axis
        flip_angle_refocus = flip_angle_refocus_deg * units.deg
        pulse_refocus = sycomore.bloch.pulse(flip_angle_refocus, phase=np.pi*units.rad/2) 
        # apply pulse
        M = update_and_record_magnetization(M, record, t, update=pulse_refocus)

    time, magnetization = list(zip(*record))

    magnetization = np.array(magnetization).round(3)

    magnetization_predicted_pre = calculate_magnetization(time[:-1], species, flip_angle)
    # pass magnetization right before the refocusing pulse
    magnetization_predicted_post = calculate_magnetization_pre_refocusing(magnetization[-2], species, flip_angle_refocus)

    magnetization_predicted = np.concatenate((magnetization_predicted_pre, magnetization_predicted_post[None]))

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.plot(time, magnetization[:, 0], label="Species $M_x$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,0], label="Species $M_x$ Predicted", linestyle='--', color='#F25050')
    #plt.plot(time, magnetization[:, 2], label="Species $M_z$", linestyle='dotted', color='#BF1515')
    #plt.plot(time, magnetization_predicted[:,2], label="Species $M_z$ Predicted", linestyle='dashdot', color='#F2668B')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.subplot(122)
    plt.plot(time, magnetization[:, 1], label="Species $M_y$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,1], label="Species $M_y$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.tight_layout()
    plt.show()

Notice below some obvious but important facts: a) because the pulse is applied along the y-axis, the vector component along the y-axis is unchanged; 2) refocusing can still occur for angles other than 180. However, note the effect on the overall amplitude. 

In [9]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1.1, description='Step size (ms)'),
    flip_angle_refocus_deg=widgets.FloatSlider(min=1, max=180, step=1, value=180, description='FA 2 (deg)')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, flip_angle_refocus_deg=180)>

## Part 3: Evolution after refocusing pulse for a single spin isochromat

In [10]:
def calculate_magnetization_post_refocusing(M_post, time_ms, tau, species):

    # Magnetization before refocusing pulse 
    Mx_post, My_post, Mz_post = M_post    

    t = np.array(time_ms) - tau

    dw =  - 2 * np.pi * species.delta_omega.convert_to(units.Hz)

    E1 = np.exp(-t/species.T1.convert_to(units.ms))
    E2 = np.exp(-t/species.T2.convert_to(units.ms))

    cos = np.cos(dw * t * 1e-3)
    sin = np.sin(dw * t * 1e-3)

    Mx = E2 * (Mx_post * cos + My_post * sin)
    My = E2 * (-Mx_post * sin + My_post * cos)
    Mz = (1 - E1) + Mz_post * E1 
    
    return np.stack((Mx, My, Mz), -1)

In [11]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, flip_angle_refocus_deg=180):
    
    # define species of spin isochromat
    # ---------------------------------

    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species = get_species(T1, T2, delta_omega=-delta_omega)

    # initialize the spin state 
    t = 0 * units.ms
    M = np.array([0, 0, 1, 1])
    record = [[t.convert_to(units.ms), M[:3] / M[3]]]

    # define experiment bloacks
    # -------------------------

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time
    idle = sycomore.bloch.time_interval(species, step_size) 

    # excitation pulse, phase implies it is along x-axis. Vector rotates about x-axis. If flip angle is 90 degrees, vector flip to +y-axis
    flip_angle = flip_angle_deg * units.deg
    pulse = sycomore.bloch.pulse(flip_angle, phase=np.pi*units.rad) 

    # define experiment blocks
    # ------------------------

    # apply pulse
    M = update_and_record_magnetization(M, record, t, update=pulse)

    # run 100 steps 
    for _ in range(100):
        t += step_size
        M = update_and_record_magnetization(M, record, t, update=idle)

    time, magnetization = list(zip(*record))

    # refocusing
    # ----------

    # apply (e.g., 180) pulse along y direction (specified by phase), only spins off-phase rotate above y-axis
    flip_angle_refocus = flip_angle_refocus_deg * units.deg
    pulse_refocus = sycomore.bloch.pulse(flip_angle_refocus, phase=np.pi*units.rad/2) 
    # apply pulse
    tau = t.convert_to(units.ms)
    M = update_and_record_magnetization(M, record, t, update=pulse_refocus)

    # let's get the magnetization up to this point here
    time, magnetization = list(zip(*record))

    magnetization = np.array(magnetization).round(3)

    magnetization_predicted_pre = calculate_magnetization(time[:-1], species, flip_angle)
    # pass magnetization right before the refocusing pulse
    magnetization_predicted_post = calculate_magnetization_pre_refocusing(magnetization[-2], species, flip_angle_refocus)

    magnetization_predicted = np.concatenate((magnetization_predicted_pre, magnetization_predicted_post[None]))
    

    # run another 100 steps 
    for _ in range(100):
        t += step_size
        M = update_and_record_magnetization(M, record, t, update=idle)

    # let's get the magnetization up to this point here
    time, magnetization = list(zip(*record))
    
    magnetization = np.array(magnetization).round(3)
    
    # get magnetization for the 100 steps after the refocusing pulse
    magnetization_post_refocus = calculate_magnetization_post_refocusing(magnetization_predicted_post, time[-100:], tau, species)

    magnetization_predicted = np.concatenate((magnetization_predicted, magnetization_post_refocus))

    plt.figure(figsize=(10,5))
    plt.subplot(121)
    plt.plot(time, magnetization[:, 0], label="Species $M_x$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,0], label="Species $M_x$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, magnetization[:, 2], label="Species $M_z$", linestyle='dotted', color='#BF1515')
    plt.plot(time, magnetization_predicted[:,2], label="Species $M_z$ Predicted", linestyle='dashdot', color='#F2668B')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.subplot(122)
    plt.plot(time, magnetization[:, 1], label="Species $M_y$", color='#6AD991')
    plt.plot(time, magnetization_predicted[:,1], label="Species $M_y$ Predicted", linestyle='--', color='#F25050')
    plt.plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species $M_\perp$", color='black')
    plt.ylim(-1,1)
    plt.legend()

    plt.tight_layout()
    plt.show()

Note that, although the direction of the y-component doesn't change after we apply the refocusing pulse, the direction is reversed. 

In [12]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=100, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1.1, description='Step size (ms)'),
    flip_angle_refocus_deg=widgets.FloatSlider(min=1, max=180, step=1, value=180, description='FA 2 (deg)')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, flip_angle_refocus_deg=180)>

## Part 4: Spin behavior of two spin isochromats

So far we have dealt with a single isochromat to compare simulated and theoretial spin behavior. We will now explore the behavior of two spins.

In [13]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg):
    
    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=6.283)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species_A = get_species(T1, T2, delta_omega=-delta_omega)
    species_B = get_species(T1, T2, delta_omega=delta_omega)

    flip_angle = flip_angle_deg * units.deg

    # temporal resolution of experiment 
    step_size = 10 * units.ms 

    # nothing occurs during this time
    idle_A = sycomore.bloch.time_interval(species_A, step_size)
    idle_B = sycomore.bloch.time_interval(species_B, step_size) 

    pulse = sycomore.bloch.pulse(flip_angle)

    # initialize the spins
    t = 0 * units.s

    M_A = np.array([0, 0, 1, 1])
    M_B = np.array([0, 0, 1, 1])

    record_A = [[t.convert_to(units.ms), M_A[:3] / M_A[3]]]
    record_B = [[t.convert_to(units.ms), M_B[:3] / M_B[3]]]

    # Update and record magnetization for the first 10 steps
    for _ in range(10):
        t += 10 * units.ms
        M_A = update_and_record_magnetization(M_A, record_A, t, update=idle_A)
        M_B = update_and_record_magnetization(M_B, record_B, t, update=idle_B)

    # Apply the pulse and record the magnetization
    M_A = update_and_record_magnetization(M_A, record_A, t, update=pulse)
    M_B = update_and_record_magnetization(M_B, record_B, t, update=pulse)
    # Update and record magnetization for the next 100 steps
    for _ in range(100):
        t += 10 * units.ms
        
        M_A = update_and_record_magnetization(M_A, record_A, t, update=idle_A)
        M_B = update_and_record_magnetization(M_B, record_B, t, update=idle_B)

    time, magnetization_A = list(zip(*record_A))
    time, magnetization_B = list(zip(*record_B))
    magnetization_A = np.array(magnetization_A)
    magnetization_B = np.array(magnetization_B)
    magnetization_C = (magnetization_A + magnetization_B) / 2.0

    fig, ax = plt.subplots(1,2,figsize=(20,10))
    ax[0].plot(time, np.linalg.norm(magnetization_A[:, :2], axis=-1), label="Species A $M_\perp$")
    ax[0].plot(time, magnetization_A[:, 2], label="Species A $M_z$")
    ax[0].plot(time, np.linalg.norm(magnetization_B[:, :2], axis=-1), label="Species B $M_\perp$", linestyle='--')
    ax[0].plot(time, magnetization_B[:, 2], label="Species B $M_z$", linestyle='--')
    ax[0].plot(time, np.linalg.norm(magnetization_C[:, :2], axis=-1), label="Species C $M_\perp$", linestyle='--')
    ax[0].plot(time, magnetization_C[:, 2], label="Species C $M_z$", linestyle='--')
    ax[0].set_xlim(0)

    ax[0].set_xlabel("Time (ms)")
    ax[0].set_ylabel("$M/M_0$")


    for time_id in range(10, 30):
        
        ax[1].plot([0, magnetization_A[time_id, 0]], [0, magnetization_A[time_id, 1]], color='black')
        ax[1].plot([0, magnetization_B[time_id, 0]], [0, magnetization_B[time_id, 1]], color='red')
        
        if time_id==11: 
            ax[1].text(magnetization_A[time_id, 0]+0.05, magnetization_A[time_id, 1]-0.1, 't = %.2f ms'%(time[time_id]), fontsize=18)
        if time_id==21: 
            ax[1].text(magnetization_A[time_id, 0]+0.05, magnetization_A[time_id, 1]-0.1, 't = %.2f ms'%(time[time_id]), fontsize=18)
        if time_id==29: 
            ax[1].text(magnetization_A[time_id, 0], magnetization_A[time_id, 1]+0.1, 't = %.2f ms'%(time[time_id]), fontsize=18)
        
    ax[1].annotate("", xy=(0, -1.25), xytext=(0, 0),
                arrowprops=dict(arrowstyle="->"))
    ax[1].annotate("", xy=(1.25, 0), xytext=(0, 0),
                arrowprops=dict(arrowstyle="->"))

    plt.legend(["Species A $M_\perp$", "Species B $M_\perp$"])
    plt.ylim(-1.5, 1.5)
    plt.xlim(-1.5, 1.5)
    plt.show()


In the figure below, the magnitude of the transverse magnetization of each species has the same decay. However, experimentally, we measure the combined signal of all spins. Note that the combined transverse magnetization has much faster decay. Note the time where the signal reaches a local minima, and where it reaches a local maxima. These are spin echos. Let's visualize the rotation of each spin to further understand this behavior. 

In [14]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=600, step=1, value=100, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg)>

## Part 4: Refocusing pulse for two spin isochromats

So far we have dealt with a single isochromat to compare simulated and theoretial spin behavior. We will now explore the behaviro for two spins: 

In [15]:
def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_mm, n_species):
    
    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=1)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms

    species_ensemble = [get_species(T1, T2, delta_omega=delta_omega*np.random.randn()) for _ in range(n_species)]

    flip_angle = flip_angle_deg * units.deg

    # temporal resolution of experiment 
    step_size = step_size_mm * units.ms 
    
    # nothing occurs during this time
    idles = [sycomore.bloch.time_interval(species, step_size) for species in species_ensemble]

    pulse = sycomore.bloch.pulse(flip_angle)

    # initialize the spins
    t = 0 * units.s

    M = [np.array([0, 0, 1, 1]) for _ in range(n_species)]

    records = [[[t.convert_to(units.ms), m[:3] / m[3]]] for m in M]

    # Update and record magnetization for the first 10 steps
    for _ in range(10):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])

    for k in range(n_species):
        M[k] = update_and_record_magnetization(M[k], records[k], t, update=pulse)

    # Update and record magnetization for the next 100 steps
    for _ in range(100):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])

  #  records = np.stack(records)

    return records

In [16]:
def calculate_magnetization_post_refocusing(M_post, time_ms, tau, species):

    # Magnetization before refocusing pulse 
    Mx_post, My_post, Mz_post = M_post    

    t = np.array(time_ms) - tau

    dw =  - 2 * np.pi * species.delta_omega.convert_to(units.Hz)

    E1 = np.exp(-t/species.T1.convert_to(units.ms))
    E2 = np.exp(-t/species.T2.convert_to(units.ms))

    cos = np.cos(dw * t * 1e-3)
    sin = np.sin(dw * t * 1e-3)

    Mx = E2 * (Mx_post * cos + My_post * sin)
    My = E2 * (-Mx_post * sin + My_post * cos)
    Mz = (1 - E1) + Mz_post * E1 
    
    return np.stack((Mx, My, Mz), -1)

def run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, n_species):
    
    delta_omega = get_inhomogeneity(B0=3, inhomogeneity_ppm=-0.2)

    T1 = T1_ms * units.ms
    T2 = T2_ms * units.ms
    n_species = int(n_species)

    species_baseline = [get_species(T1, T2, delta_omega=-delta_omega)]
    species_ensemble = species_baseline + [get_species(T1, T2, delta_omega=delta_omega*np.random.randn()) for _ in range(n_species-1)]

    # temporal resolution of experiment 
    step_size = step_size_ms * units.ms 

    # nothing occurs during this time
    idles = [sycomore.bloch.time_interval(species, step_size) for species in species_ensemble]

    pulse = sycomore.bloch.pulse(flip_angle_deg * units.deg, phase=np.pi*units.rad)

    flip_angle_refocus_deg = 180
    pulse_refocus = sycomore.bloch.pulse(flip_angle_refocus_deg * units.deg, phase=np.pi*units.rad/2) 

    # initialize the spins
    t = 0 * units.s

    M = [np.array([0, 0, 1, 1]) for _ in range(n_species)]

    records = [[[t.convert_to(units.ms), m[:3] / m[3]]] for m in M]

    for k in range(n_species):
        M[k] = update_and_record_magnetization(M[k], records[k], t, update=pulse)

    # Update and record magnetization for the next 100 steps
    for _ in range(100):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])

    # apply 180 degree refocusing pulse
    tau = t.convert_to(units.ms)
    for k in range(n_species):
        M[k] = update_and_record_magnetization(M[k], records[k], t, update=pulse_refocus)

    # Update and record magnetization for the next 100 steps
    for _ in range(300):
        t += step_size
        for k in range(n_species):
            M[k] = update_and_record_magnetization(M[k], records[k], t, update=idles[k])
            
    magnetization = []
    #magnetization_predicted = []
    for species, record in zip(species_ensemble[::-1], records[::-1]):
        time, magnetization_k = list(zip(*record))
        magnetization_k = np.array(magnetization_k)

        #M_pred = calculate_magnetization_post_refocusing(M_post=magnetization_k[-300], time_ms=time[-300:], tau=tau, species=species)
        #magnetization_predicted.append(M_pred)
        magnetization.append(np.array(magnetization_k))

    magnetization = np.stack(magnetization).mean(axis=0)
    #magnetization_predicted = np.stack(magnetization_predicted).mean(axis=0)

    

    fig, ax = plt.subplots(1,2,figsize=(20,10))
    fontsize = 20
    for label, j in zip(['M_x', 'M_y'], [0, 1]):
        ax[j].plot(time, np.linalg.norm(magnetization_k[:, :2], axis=-1), label="Species k $M_\perp$", color='black', linewidth=4)
        ax[j].plot(time, np.linalg.norm(magnetization[:, :2], axis=-1), label="Species Ensemble $M_\perp$", color='#F25050', linewidth=4, linestyle='--')
        #ax[j].plot(time[-300:], np.linalg.norm(magnetization_predicted[:, :2], axis=-1), label="Species Ensemble Pred $M_\perp$", color='blue', linewidth=4, linestyle='--')
        ax[j].plot(time, magnetization_k[:, j], label="Species k $%s$"%(label), color='black', linewidth=1)
        ax[j].plot(time, magnetization[:, j], label="Species Ensemble $%s$"%(label), color='#F25050', linewidth=1, linestyle='--')
        #ax[j].plot(time[-300:], magnetization_predicted[:, j], label="Species Ensemble Pred $%s$"%(label), color='blue', linewidth=1, linestyle='--')
        ax[j].axvline(x=tau, color='gray', linestyle='--', alpha=0.5)
        ax[j].axvline(x=tau*2, color='gray', linestyle='--', alpha=0.5)
        ax[j].text(tau, -0.8, 'TE/2', fontsize=fontsize)
        ax[j].text(tau*2, -0.8, 'TE', fontsize=fontsize)
        ax[j].set_xlabel("Time (ms)", fontsize=fontsize)
        ax[j].set_ylabel("$M/M_0$", fontsize=fontsize)
        ax[j].legend(loc='best', fontsize='large')
        ax[j].set_ylim(-1,1)

    plt.show()

In [17]:
widgets.interact(
    run_simulation,
    T1_ms=widgets.FloatSlider(min=100, max=2000, step=10, value=1000, description='T1 (ms)'),
    T2_ms=widgets.FloatSlider(min=10, max=200, step=1, value=88, description='T2 (ms)'),
    flip_angle_deg=widgets.FloatSlider(min=0, max=180, step=1, value=90, description='FA 1 (deg)'),
    step_size_ms=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.7, description='Step size (ms)'),
    n_species=widgets.FloatSlider(min=1, max=1000, step=1, value=1000, description='n spins')
)

interactive(children=(FloatSlider(value=1000.0, description='T1 (ms)', max=2000.0, min=100.0, step=10.0), Floa…

<function __main__.run_simulation(T1_ms, T2_ms, flip_angle_deg, step_size_ms, n_species)>